### Setup & Install Dependencies

In [ ]:
!pip install langchain sentence-transformers faiss-cpu chromadb ollama jupyter

### Import Libraries

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import Ollama
from langchain.chains import RetrievalQA

### Load Your Knowledge Base

In [ ]:
docs = [
    {"title": "RAG", "text": "Retrieval-Augmented Generation (RAG) combines retrieval with LLMs..."},
    {"title": "LangChain", "text": "LangChain is a Python framework for building LLM applications..."}
]
texts = [d["text"] for d in docs]

### Chunk & Embed Documents

In [ ]:
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = splitter.create_documents(texts)

embedder = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(chunks, embedder)

### Setup Retriever + LLM

In [ ]:
retriever = db.as_retriever(search_kwargs={"k":2})
llm = Ollama(model="mistral")  # or "deepseek-r1:14b"

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff"
)

### Ask Questions

In [ ]:
query = "What is RAG?"
result = qa_chain.run(query)
print("Q:", query)
print("A:", result)

### Add a loop for multiple queries

In [ ]:
while True:
    query = input("Ask a question (or 'exit'): ")
    if query.lower() == "exit":
        break
    print(qa_chain.run(query))